# init

In [1]:
from quantifiers.CC import classify_count
from quantifiers.ACC import ACC
from quantifiers.dys_method import dys_method
from quantifiers.MS import MS_method

from utils.getTrainingScores import getTrainingScores
from utils.getTPRFPR import getTPRFPR
from utils.applyquantifiers import apply_quantifier
from utils.fitQuantifierSchumacherGithub import fitQuantifierSchumacherGithub

import pdb
import quapy as qp
import os
import numpy as np
import pandas as pd
import joblib
from scipy.io.arff import loadarff
from pprint import pprint
import matplotlib.pyplot as plt
import time
import math

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import CalibrationDisplay

meta_table_path = './metafeatures/meta-table.csv'
path = "./datasets/"
files = os.listdir(path)


counters = ['CC', 'ACC', 'PACC', 'PCC', 'SMM', 'HDy', 'DyS', 'SORD', 'MS', 'MS2', 'MAX', 'X', 'T50']
# counters = ["CC","ACC","SMM","HDy","DyS","SORD","MS","MS2","MAX","X","T50","PCC","PACC","GAC","GPAC","FM"]

# preprocess

In [2]:
i = 0
dataframe = None
X = None
y = None
X_list = []
y_list = []

for f in files:
  df = pd.read_csv(path + f)
  df = df.dropna()
  
  y = df.pop(df.columns[-1])
  X = df

  y_list.append(y.to_numpy())
  X_list.append(X.to_numpy())

  i += 1
i = 0

In [3]:
meta_table = pd.read_csv(meta_table_path)
meta_table_dict = {}

for counter in counters:
    meta_table_dict[counter] = meta_table.copy(deep = True)

meta_table

,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,can_cor.mean,can_cor.sd,cat_to_num,...,tree_depth.sd,tree_imbalance.mean,tree_imbalance.sd,tree_shape.mean,tree_shape.sd,var_importance.mean,var_importance.sd,w_lambda,worst_node.mean,worst_node.sd
0,0.008139,0.043889,0.010032,0.490599,0.004135,0.561303,0.002908,0.561218,0.0,0.0,...,0.313957,0.018766,0.337663,0.012390,0.179874,0.038708,0.069519,0.647064,0.556616,0.003278
1,0.106477,0.194464,0.433981,0.412370,0.008805,0.204547,0.454586,0.279843,0.0,0.0,...,0.144613,0.120941,0.580063,0.100662,0.342257,0.246988,0.132723,0.890612,0.189155,0.237606
2,0.116658,0.341414,0.529407,0.470566,0.010323,0.784169,0.010684,0.528722,0.0,0.0,...,0.234904,0.074992,0.484229,0.099970,0.413085,0.050255,0.085777,0.682031,0.785998,0.012040
3,0.137671,0.419773,0.557749,0.391798,0.009623,0.819016,0.001322,0.394100,0.0,0.0,...,0.185417,0.070290,0.443044,0.071106,0.380437,0.050255,0.041365,0.807865,0.821864,0.001490
4,0.276312,0.343908,0.366152,0.208240,0.099335,0.431910,0.464158,0.602796,0.0,0.0,...,0.179912,0.115313,0.631564,0.212819,0.834952,0.047472,0.055058,0.599718,0.409391,0.189726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,0.074462,0.356852,0.204001,0.657376,0.001311,0.729914,0.244046,0.826111,0.0,0.0,...,0.247767,0.038638,0.341897,0.010386,0.094861,0.013001,0.070563,0.295391,0.486346,0.098109
157,0.098333,0.139054,0.879873,0.633259,0.000006,0.980050,0.035743,0.927209,0.0,0.0,...,0.196475,0.079318,0.411869,0.053137,0.305895,0.397590,0.838834,0.129877,0.561986,0.211390
158,0.064769,0.067070,0.831128,0.078607,0.000286,0.564553,0.075999,0.452016,0.0,0.0,...,0.209878,0.043389,0.303553,0.022339,0.166031,0.397590,0.168393,0.757488,0.534034,0.044873
159,0.057703,0.277545,0.130624,0.565499,0.004991,0.596928,0.234005,0.725345,0.0,0.0,...,0.396308,0.020108,0.351968,0.031128,0.271067,0.031213,0.082105,0.443156,0.301793,0.065641


# meta-features

In [4]:
# def ARR(dataset, algorithm):
  

def run_experiment(X, y, dataset_name):
  #......................input/output path directories....................

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
  
  rf_clf = None
  rf_clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
  
  calib_clf = CalibratedClassifierCV(rf_clf, cv=3, n_jobs=-1)
  calib_clf.fit(X_train, y_train)

  # try:
  #   rf_clf = joblib.load('./models/' + dataset_name + '.joblib')
  #   rf_clf.n_jobs = -1
  # except:
  #   rf_clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

  scores = getTrainingScores(X_train, y_train, 10, rf_clf)[0] # None
  tprfpr = getTPRFPR(scores)
  rf_clf.fit(X_train, y_train)

  # disp = CalibrationDisplay.from_estimator(rf_clf, X_test, y_test)
  # disp.plot()

  # disp = CalibrationDisplay.from_estimator(calib_clf, X_test, y_test)
  # disp.plot()

  # plt.show()

  niterations = 10 # how many replicates it will take
  batch_sizes = list([50]) # list(range(10, min(91, max_allowed + 1), 10))# + list(range(100, min(501, max_allowed + 1), 100)) # test set sizes
  alpha_values = [round(x, 2) for x in np.linspace(0,1,20)]   # class proportion

  pos_scores = scores[scores["class"]==1]["scores"]
  neg_scores = scores[scores["class"]==0]["scores"]
  
  X_test = pd.DataFrame(X_test)
  y_test = pd.DataFrame(y_test, columns=[str(len(X_test.columns))])
  df_test = pd.concat([X_test, y_test], axis=1)
  
  # WAS ZERO (0) BEFORE
  df_test_pos = df_test.loc[df_test[df_test.columns[-1]] == 1] # seperating positive test examples
  df_test_neg = df_test.loc[df_test[df_test.columns[-1]] == 0] # seperating negative test examples
  
  # pdb.set_trace()

  table=pd.DataFrame(columns=['quantifier', 'abs-error', 'execution-time'])
  for sample_size in batch_sizes:   # [10,100,500], batch_sizes, Varying test set sizes

    for alpha in alpha_values: # Varying positive class distribution
      # abs_error_cc = []
      # abs_error_dys = []

      error = []

      for iter in range(niterations):
        pos_size = int(round(sample_size * alpha, 2))
        neg_size = sample_size - pos_size

        #
        # AVISAR O PROF ANDRE QUE SÓ FUNCIONOU COM REPLACE = TRUE
        #
        # df_test_neg
        # sample_test_pos = df_test_pos.sample( int(pos_size), replace = False)
        # sample_test_neg = df_test_neg.sample( int(neg_size), replace = False)
        sample_test_pos = df_test_pos.sample( int(pos_size), replace = True)
        sample_test_neg = df_test_neg.sample( int(neg_size), replace = True)

        sample_test = pd.concat([sample_test_pos, sample_test_neg])

        test_label = sample_test[sample_test.columns[-1]] # sample_test["class"]

        test_sample = sample_test.drop([sample_test.columns[-1]], axis=1) # sample_test.drop(["class"], axis=1)  #dropping class label columns
        te_scores = rf_clf.predict_proba(test_sample)[:,1]  #estimating test sample scores

        n_pos_sample_test = list(test_label).count(1) #Counting num of actual positives in test sample
        calcultd_pos_prop = round(n_pos_sample_test/len(sample_test), 2) #actual pos class prevalence in generated sample

        # print(counters)
        for quantifier in counters:
          #..............Test Sample QUAPY exp...........................
          te_quapy = None
          external_qnt = None
          #if quantifier in ['EM', 'PWK']:
          #  print('ok')
          #  external_qnt = fitQuantifierSchumacherGithub(quantifier, X_train, y_train)                  
          #  te_quapy = qp.data.LabelledCollection(sample_test.drop(["class","Binary_label"], axis=1), test_label)
          
          #.............Calling of Methods..................................................
          start = time.time()
          pred_pos_prop = apply_quantifier(qntMethod=quantifier,
                                           clf=calib_clf,
                                           scores=scores,
                                           p_score=pos_scores,
                                           n_score=neg_scores,
                                           train_labels=None,
                                           test_score=te_scores,
                                           TprFpr=tprfpr,
                                           thr=0.5,
                                           measure='hellinger',
                                           test_data=test_sample,
                                           test_quapy=te_quapy,
                                           external_qnt=external_qnt) #y_test=test_label
          stop = time.time()
          t = stop - start

          pred_pos_prop = np.round(pred_pos_prop,2)  #predicted class proportion
          
          #..............................RESULTS Evaluation.....................................
          abs_error = round(abs(calcultd_pos_prop - pred_pos_prop), 2) # absolute error
          
          table.loc[len(table)] = [quantifier,abs_error,t]
          # error = round(calcultd_pos_prop - pred_pos_prop, 2)     # simple error Biasness
          
  return table

In [5]:
clf = RandomForestClassifier(random_state=42)
skip_count = 0
table = None
# warnings.filterwarnings('ignore')

result = {}
for counter in counters:
    result[counter] = []

file = open('log.txt', 'w')
file.close()

for i in range(0, len(X_list)):
  try:
    clf = RandomForestClassifier()

    table = run_experiment(X_list[i], y_list[i], str(files[i].split('.csv')[0]))

    # print(table)

    table = table.groupby('quantifier')['abs-error', 'execution-time'].aggregate('mean')

    # pdb.set_trace()

    alpha = 0
    for key in counters:
      sum = 0
      for k in counters:
        if k != key:
          sum += ((1 - table['abs-error'][key]) / (1 - table['abs-error'][k])) / (1 + alpha * math.log((table['execution-time'][key] / table['execution-time'][k])))
      # pdb.set_trace()
      arr = sum / (len(counters) - 1)
      result[key].append(arr)

    # pdb.set_trace()
 
    print('Finished ' + str(i))
  except Exception as e:
    print('Skipping ' + str(i) + '...\t\t\t' + str(e))
    # for key in meta_table_dict:
    #   meta_table_dict[key].drop(i, inplace = True)
    result[key].append(-1)
    skip_count += 1

    file = open('log.txt', 'a')
    file.write('Skipping ' + str(i) + '...\t\t\t' + str(e) + '\n')
    file.write('table[abs-error][key]:\n' + str(table['abs-error'][key]) + '\n')
    file.write('table[abs-error][k]:\n' + str(table['abs-error'][k]) + '\n')
    file.write('table[execution-time][key]:\n' + str(table['execution-time'][key]) + '\n')
    file.write('table[execution-time][k]:\n' + str(table['execution-time'][k]) + '\n')
    file.write(str(table['execution-time'][key] / table['execution-time'][k]) + '\n')
    file.close()

  # # TEST
  # if i == 3:
  #   for key in meta_table_dict:
  #     meta_table_dict[key].reset_index(drop=True, inplace=True)
  #     meta_table_dict[key].drop(meta_table_dict[key].index[((i+1)-skip_count):], inplace=True)
  #   break

# for key in meta_table_dict:
#   meta_table_dict[key]['arr'] = result[key]

for key in result:
    meta_table[('arr_' + key)] = result[key]

print('\n\nSkipped ' + str(skip_count) + ' dataset(s)')

  Fold #1
Training_len 3193
SCORES_Length: 320
  Fold #2
Training_len 3193
SCORES_Length: 640
  Fold #3
Training_len 3193
SCORES_Length: 960
  Fold #4
Training_len 3193
SCORES_Length: 1279
  Fold #5
Training_len 3193
SCORES_Length: 1598
  Fold #6
Training_len 3193
SCORES_Length: 1917
  Fold #7
Training_len 3193
SCORES_Length: 2236
  Fold #8
Training_len 3193
SCORES_Length: 2555
  Fold #9
Training_len 3193
SCORES_Length: 2874
  Fold #10
Training_len 3193
SCORES_Length: 3193
Finished 0
  Fold #1
Training_len 258
SCORES_Length: 26
  Fold #2
Training_len 258
SCORES_Length: 52
  Fold #3
Training_len 258
SCORES_Length: 78
  Fold #4
Training_len 258
SCORES_Length: 104
  Fold #5
Training_len 258
SCORES_Length: 130
  Fold #6
Training_len 258
SCORES_Length: 156
  Fold #7
Training_len 258
SCORES_Length: 182
  Fold #8
Training_len 258
SCORES_Length: 208
  Fold #9
Training_len 258
SCORES_Length: 233
  Fold #10
Training_len 258
SCORES_Length: 258
Finished 1
  Fold #1
Training_len 1020
SCORES_Length:

d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) 

Finished 10
  Fold #1
Training_len 282
SCORES_Length: 29
  Fold #2
Training_len 282
SCORES_Length: 58
  Fold #3
Training_len 282
SCORES_Length: 86
  Fold #4
Training_len 282
SCORES_Length: 114
  Fold #5
Training_len 282
SCORES_Length: 142
  Fold #6
Training_len 282
SCORES_Length: 170
  Fold #7
Training_len 282
SCORES_Length: 198
  Fold #8
Training_len 282
SCORES_Length: 226
  Fold #9
Training_len 282
SCORES_Length: 254
  Fold #10
Training_len 282
SCORES_Length: 282
Finished 11
  Fold #1
Training_len 191
SCORES_Length: 20
  Fold #2
Training_len 191
SCORES_Length: 39
  Fold #3
Training_len 191
SCORES_Length: 58
  Fold #4
Training_len 191
SCORES_Length: 77
  Fold #5
Training_len 191
SCORES_Length: 96
  Fold #6
Training_len 191
SCORES_Length: 115
  Fold #7
Training_len 191
SCORES_Length: 134
  Fold #8
Training_len 191
SCORES_Length: 153
  Fold #9
Training_len 191
SCORES_Length: 172
  Fold #10
Training_len 191
SCORES_Length: 191
Finished 12
  Fold #1
Training_len 91
SCORES_Length: 10
  Fold

d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) 

Finished 29
  Fold #1
Training_len 70
SCORES_Length: 7
  Fold #2
Training_len 70
SCORES_Length: 14
  Fold #3
Training_len 70
SCORES_Length: 21
  Fold #4
Training_len 70
SCORES_Length: 28
  Fold #5
Training_len 70
SCORES_Length: 35
  Fold #6
Training_len 70
SCORES_Length: 42
  Fold #7
Training_len 70
SCORES_Length: 49
  Fold #8
Training_len 70
SCORES_Length: 56
  Fold #9
Training_len 70
SCORES_Length: 63
  Fold #10
Training_len 70
SCORES_Length: 70
Finished 30
  Fold #1
Training_len 4282
SCORES_Length: 429
  Fold #2
Training_len 4282
SCORES_Length: 858
  Fold #3
Training_len 4282
SCORES_Length: 1286
  Fold #4
Training_len 4282
SCORES_Length: 1714
  Fold #5
Training_len 4282
SCORES_Length: 2142
  Fold #6
Training_len 4282
SCORES_Length: 2570
  Fold #7
Training_len 4282
SCORES_Length: 2998
  Fold #8
Training_len 4282
SCORES_Length: 3426
  Fold #9
Training_len 4282
SCORES_Length: 3854
  Fold #10
Training_len 4282
SCORES_Length: 4282
Finished 31
  Fold #1
Training_len 848
SCORES_Length: 85


...

C:\Users\guilh\AppData\Local\Temp\ipykernel_21624\2419100764.py:30: RuntimeWarning: divide by zero encountered in double_scalars
  sum += ((1 - table['abs-error'][key]) / (1 - table['abs-error'][k])) / (1 + alpha * math.log((table['execution-time'][key] / table['execution-time'][k])))


Skipping 101...			math domain error
  Fold #1
Training_len 387
SCORES_Length: 39
  Fold #2
Training_len 387
SCORES_Length: 78
  Fold #3
Training_len 387
SCORES_Length: 117
  Fold #4
Training_len 387
SCORES_Length: 156
  Fold #5
Training_len 387
SCORES_Length: 195
  Fold #6
Training_len 387
SCORES_Length: 234
  Fold #7
Training_len 387
SCORES_Length: 273
  Fold #8
Training_len 387
SCORES_Length: 311
  Fold #9
Training_len 387
SCORES_Length: 349
  Fold #10
Training_len 387
SCORES_Length: 387
Finished 102
  Fold #1
Training_len 186
SCORES_Length: 19
  Fold #2
Training_len 186
SCORES_Length: 38
  Fold #3
Training_len 186
SCORES_Length: 57
  Fold #4
Training_len 186
SCORES_Length: 76
  Fold #5
Training_len 186
SCORES_Length: 95
  Fold #6
Training_len 186
SCORES_Length: 114
  Fold #7
Training_len 186
SCORES_Length: 132
  Fold #8
Training_len 186
SCORES_Length: 150
  Fold #9
Training_len 186
SCORES_Length: 168
  Fold #10
Training_len 186
SCORES_Length: 186
Finished 103
Skipping 104...			could

d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) / diff_tpr_fpr
d:\Documentos\Code\Mestrado\Code\quantifier-recommender\quantifiers\PACC.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  pos_prop = (pos_prop - float(TprFpr['fpr'])) 

Skipping 126...			math domain error
  Fold #1
Training_len 175
SCORES_Length: 18
  Fold #2
Training_len 175
SCORES_Length: 36
  Fold #3
Training_len 175
SCORES_Length: 54
  Fold #4
Training_len 175
SCORES_Length: 72
  Fold #5
Training_len 175
SCORES_Length: 90
  Fold #6
Training_len 175
SCORES_Length: 107
  Fold #7
Training_len 175
SCORES_Length: 124
  Fold #8
Training_len 175
SCORES_Length: 141
  Fold #9
Training_len 175
SCORES_Length: 158
  Fold #10
Training_len 175
SCORES_Length: 175
Finished 127
Skipping 128...			could not convert string to float: 'Female'
Skipping 129...			could not convert string to float: 'A'
Skipping 130...			could not convert string to float: '60-74'
  Fold #1
Training_len 105
SCORES_Length: 11
  Fold #2
Training_len 105
SCORES_Length: 22
  Fold #3
Training_len 105
SCORES_Length: 33
  Fold #4
Training_len 105
SCORES_Length: 44
  Fold #5
Training_len 105
SCORES_Length: 55
  Fold #6
Training_len 105
SCORES_Length: 65
  Fold #7
Training_len 105
SCORES_Length: 75


ValueError: Length of values (149) does not match length of index (161)

In [ ]:
meta_table.to_csv('./metafeatures/meta-table-processed.csv', index = False)

In [6]:
result

{'CC': [0.9367764233855486,
  1.0376313359469556,
  0.8510307879797064,
  0.7483043918393822,
  -1,
  0.8813827357742393,
  1.0264640082719305,
  -1,
  1.0789126001910758,
  0.8106752135916375,
  0.7994693668124854,
  0.8611147787976873,
  1.0751637118883521,
  0.8253133843771452,
  1.1133088496471484,
  1.0874049025589423,
  1.0042579833754879,
  1.043933968627514,
  1.0696286939785515,
  1.049581808510961,
  1.0567255847107062,
  1.0850031198989025,
  0.9762396677999116,
  0.9358851016650563,
  1.0856865087189174,
  0.7419452281389914,
  0.916480854352015,
  1.0391165921315921,
  1.16087440005098,
  0.9964076848271053,
  1.0421733462600986,
  0.708345250691309,
  0.9512585309483442,
  1.0471547074324143,
  0.9510915705347079,
  1.0052025264211815,
  1.1458819143571568,
  1.0713673810733264,
  1.0851643108591658,
  0.9545940390224787,
  1.0783852702739718,
  1.0311613016039873,
  1.0904319721602544,
  0.9368423366145433,
  1.0853957513508428,
  1.0874811119638308,
  1.0715919494597161

In [7]:
# import pickle

# with open('result.pkl', 'wb') as fp:
#     pickle.dump(result, fp)
#     print('dictionary saved successfully to file')

dictionary saved successfully to file


In [8]:
# with open('result.pkl', 'rb') as fp:
#     person = pickle.load(fp)
#     print('Person dictionary')
#     print(person)

Person dictionary
{'CC': [0.9367764233855486, 1.0376313359469556, 0.8510307879797064, 0.7483043918393822, -1, 0.8813827357742393, 1.0264640082719305, -1, 1.0789126001910758, 0.8106752135916375, 0.7994693668124854, 0.8611147787976873, 1.0751637118883521, 0.8253133843771452, 1.1133088496471484, 1.0874049025589423, 1.0042579833754879, 1.043933968627514, 1.0696286939785515, 1.049581808510961, 1.0567255847107062, 1.0850031198989025, 0.9762396677999116, 0.9358851016650563, 1.0856865087189174, 0.7419452281389914, 0.916480854352015, 1.0391165921315921, 1.16087440005098, 0.9964076848271053, 1.0421733462600986, 0.708345250691309, 0.9512585309483442, 1.0471547074324143, 0.9510915705347079, 1.0052025264211815, 1.1458819143571568, 1.0713673810733264, 1.0851643108591658, 0.9545940390224787, 1.0783852702739718, 1.0311613016039873, 1.0904319721602544, 0.9368423366145433, 1.0853957513508428, 1.0874811119638308, 1.0715919494597161, 1.0574920960080398, 1.0386278427403828, 0.9805506262074877, 1.0434622780

In [ ]:
result
# meta_table_dict

In [ ]:
meta_table

In [ ]:
# finSavePath = './metafeatures/meta-table'
# for c in counters:
#     meta_table_dict[c].to_csv(str(finSavePath + '-' + c + '.csv'), index = False)

In [ ]:
# import pandas as pd

# test = pd.read_csv('./metafeatures/meta-table-PACC.csv')
# test = test.pop(test.columns[-1])
# test

In [ ]:
# import numpy as np

# i = 0
# for t in test:
#     if np.isnan(t):
#         print(i)
#     i += 1